# Entendiendo el Informer

In [433]:
# Definir los args
args = dotdict()

args.model = 'informer' # model of experiment, options: [informer, informerstack, informerlight(TBD)]

args.data = 'train_norm' # data
args.root_path = './Dataset/Train/' # root path of data file
args.data_path = 'train_norm_full.csv' # data file
args.features = 'MS' # forecasting task, options:[M, S, MS]; M:multivariate predict multivariate, S:univariate predict univariate, MS:multivariate predict univariate
args.target = 'p95 deploy/frontend' # target feature in S or MS task
args.freq = 't' # freq for time features encoding, options:[s:secondly, t:minutely, h:hourly, d:daily, b:business days, w:weekly, m:monthly], you can also use more detailed freq like 15min or 3h
args.checkpoints = './informer_checkpoints' # location of model checkpoints

args.seq_len = 96 # input sequence length of Informer encoder
args.label_len = 48 # start token length of Informer decoder
args.pred_len = 24 # prediction sequence length
# Informer decoder input: concat[start token series(label_len), zero padding series(pred_len)]

args.enc_in = 60 # encoder input size
args.dec_in = 60 # decoder input size
args.c_out = 10 # output size
args.factor = 5 # probsparse attn factor
args.d_model = 512 # dimension of model
args.n_heads = 8 # num of heads
args.e_layers = 2 # num of encoder layers
args.d_layers = 1 # num of decoder layers
args.d_ff = 2048 # dimension of fcn in model
args.dropout = 0.05 # dropout
args.attn = 'prob' # attention used in encoder, options:[prob, full]
args.embed = 'fixed' # time features encoding, options:[timeF, fixed, learned]
args.activation = 'gelu' # activation
args.distil = True # whether to use distilling in encoder
args.output_attention = False # whether to output attention in ecoder
args.mix = True
args.padding = 0
args.scale = False

args.batch_size = 32 
args.learning_rate = 0.0001
args.loss = 'mse'
args.lradj = 'type1'
args.use_amp = False # whether to use automatic mixed precision training

args.num_workers = 0
args.itr = 1
args.train_epochs = 6
args.patience = 3
args.des = 'exp'

args.use_gpu = True if torch.cuda.is_available() else False
args.gpu = 0

args.use_multi_gpu = False
args.devices = '0,1,2,3'

## TRAIN

In [108]:
#Primero arma el traindata
import sys
import os
sys.path.append('/Users/emlanza/Library/CloudStorage/OneDrive-UniversidadAustral/Facultad/Maestria Data Science AUSTRAL/Tesis/Transformers-for-latency-prediction/Informer/Informer2020')

## DATASET

In [429]:
df_raw = pd.read_csv(os.path.join(args.root_path,
                                          args.data_path))

In [430]:
df_raw

,date,p50 deploy/adservice,p50 deploy/cartservice,p50 deploy/checkoutservice,p50 deploy/currencyservice,p50 deploy/emailservice,p50 deploy/frontend,p50 deploy/loadgenerator,p50 deploy/paymentservice,p50 deploy/productcatalogservice,...,deploy/currencyservice,deploy/emailservice,deploy/frontend,deploy/loadgenerator,deploy/paymentservice,deploy/productcatalogservice,deploy/recommendationservice,deploy/redis-cart,deploy/shippingservice,p95 deploy/frontend
0,2021-10-07 09:18:00,13.460422,-1.643790,-0.057770,-0.217764,-1.146121,-0.346990,1.110223e-16,-0.644470,-0.522305,...,-0.000017,-0.000005,-0.000005,-0.000067,-2.081668e-17,-0.000055,-0.001252,-2.081668e-17,4.805699,-0.535875
1,2021-10-07 09:18:05,10.031784,0.229134,-0.434722,-0.217764,-0.898530,-0.346990,1.110223e-16,-0.644470,-0.429260,...,-0.000017,-0.000005,-0.000005,-0.000067,-2.081668e-17,-0.000055,-0.001252,-2.081668e-17,4.805699,-0.535875
2,2021-10-07 09:18:10,10.031784,0.229134,-0.434722,-0.228521,-0.898530,-0.393260,1.110223e-16,0.922269,-0.429260,...,-0.000017,-0.000005,-0.000005,-0.000067,-2.081668e-17,-0.000055,-0.001252,-2.081668e-17,-0.005351,-0.623315
3,2021-10-07 09:18:15,8.460325,-0.890356,-0.010651,-0.228521,-0.452866,-0.393260,1.110223e-16,0.922269,-0.592773,...,-0.000017,-0.000005,-0.000005,-0.000067,-2.081668e-17,-0.000055,-0.001252,-2.081668e-17,-0.005351,-0.623315
4,2021-10-07 09:18:20,8.460325,-0.890356,-0.010651,-0.325898,-0.452866,-0.392411,1.110223e-16,0.400022,-0.592773,...,-0.000017,-0.000005,-0.000005,-0.000067,-2.081668e-17,-0.000055,-0.001252,-2.081668e-17,-0.005351,-0.237772
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1800,2021-10-07 11:48:00,-0.254131,0.002309,-0.010651,-0.722205,0.587018,-0.532381,1.110223e-16,0.086675,-0.670068,...,-0.000017,-0.000005,-0.000005,-0.000067,-2.081668e-17,-0.000055,-2.177623,-2.081668e-17,-0.005351,-0.463400
1801,2021-10-07 11:48:05,-0.254131,-0.166355,-0.434722,-0.722205,0.215631,-0.532381,1.110223e-16,0.086675,-0.586403,...,-0.000017,-0.000005,-0.000005,-0.000067,-2.081668e-17,-0.000055,-2.177623,-2.081668e-17,-0.005351,-0.463400
1802,2021-10-07 11:48:10,-0.254131,-0.166355,-0.434722,0.248105,0.215631,-0.503038,1.110223e-16,-0.748919,-0.586403,...,-0.000017,-0.000005,-0.000005,-0.000067,-2.081668e-17,-0.000055,-0.001252,-2.081668e-17,-0.005351,0.627135
1803,2021-10-07 11:48:15,0.031589,0.688184,0.554777,0.248105,-0.650939,-0.503038,1.110223e-16,-0.748919,-0.471991,...,-0.000017,-0.000005,-0.000005,-0.000067,-2.081668e-17,-0.000055,-0.001252,-2.081668e-17,-0.005351,0.627135


In [237]:
from data.data_loader import Dataset_Custom,Dataset_Pred
from utils.tools import dotdict
from exp.exp_informer import Exp_Informer
from utils.timefeatures import time_features

In [196]:
import os
import numpy as np
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
from pandas.tseries import offsets
from typing import List

In [5]:
class dotdict(dict):
    """dot.notation access to dictionary attributes"""
    __getattr__ = dict.get
    __setattr__ = dict.__setitem__
    __delattr__ = dict.__delitem__

In [441]:
# Definicion de argumentos
timeenc=0
flag='test'

In [442]:
#
Data = Dataset_Custom
timeenc = 0 if args.embed!='timeF' else 1
flag = 'train'; shuffle_flag = False; drop_last = True; batch_size = 1

data_set = Data(
            root_path=args.root_path,
            data_path=args.data_path,
            flag=flag,
            size=[args.seq_len, args.label_len, args.pred_len],
            features=args.features,
            target=args.target,
            inverse=args.inverse,
            timeenc=timeenc,
            freq=args.freq,
            cols=args.cols
        )
data_loader = DataLoader(
    data_set,
    batch_size=batch_size,
    shuffle=shuffle_flag,
    num_workers=args.num_workers,
    drop_last=drop_last)

In [443]:
data_set.data_x[1]

array([ 8.39763364e+00,  2.40214129e-01, -4.28377773e-01, -2.21491876e-01,
       -8.86168692e-01, -3.52054196e-01, -7.68083811e-01, -4.34242227e-01,
        7.17525120e-01,  1.85467071e-01,  1.79053116e-01,  1.46643220e-01,
       -3.67835180e-01, -7.31805468e-01, -1.31100546e-01, -3.03386031e-01,
        9.83190882e-01, -1.73902966e-01, -3.81574712e-01, -3.30107466e-01,
       -3.32954236e-01, -1.32440016e-01, -1.80196351e-01, -5.39782328e-01,
       -7.13910675e-01, -3.14416091e-01, -2.94456656e-01, -5.85306187e-01,
        9.73434108e-01, -1.38358969e-01, -1.92280906e-01, -5.64795317e-01,
       -3.24069510e-01, -2.27310357e-01, -5.96856376e-01, -5.02661170e-01,
       -2.16902910e-01, -6.27486143e-01, -2.15123637e-01, -5.34150667e-01,
       -8.01818576e-05, -5.23852905e-01, -5.92189081e-01, -5.00851309e-01,
       -4.74602432e-01, -5.21888083e-05, -5.15784022e-05, -1.00000000e+00,
       -1.94948064e-05, -5.62766600e-06, -5.62766603e-06, -8.01818577e-05,
       -5.09606634e-05, -

In [444]:
def check_for_nan(data_loader):
    for batch_idx, batch in enumerate(data_loader):
        # iterate over each tensor in the batch
        for tensor in batch:
            # check if the tensor contains NaN values
            if torch.isnan(tensor).any():
                print(f"Found NaN values in batch {batch_idx}")

check_for_nan(data_loader)

### Train model step by step

In [333]:
import os
import time


#### Helpers

In [334]:
class EarlyStopping:
    def __init__(self, patience=7, verbose=False, delta=0):
        self.patience = patience
        self.verbose = verbose
        self.counter = 0
        self.best_score = None
        self.early_stop = False
        self.val_loss_min = np.Inf
        self.delta = delta

    def __call__(self, val_loss, model, path):
        score = -val_loss
        if self.best_score is None:
            self.best_score = score
            self.save_checkpoint(val_loss, model, path)
        elif score < self.best_score + self.delta:
            self.counter += 1
            print(f'EarlyStopping counter: {self.counter} out of {self.patience}')
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_score = score
            self.save_checkpoint(val_loss, model, path)
            self.counter = 0

    def save_checkpoint(self, val_loss, model, path):
        if self.verbose:
            print(f'Validation loss decreased ({self.val_loss_min:.6f} --> {val_loss:.6f}).  Saving model ...')
        torch.save(model.state_dict(), path+'/'+'checkpoint.pth')
        self.val_loss_min = val_loss

In [335]:
def adjust_learning_rate(optimizer, epoch, args):
    # lr = args.learning_rate * (0.2 ** (epoch // 2))
    if args.lradj=='type1':
        lr_adjust = {epoch: args.learning_rate * (0.5 ** ((epoch-1) // 1))}
    elif args.lradj=='type2':
        lr_adjust = {
            2: 5e-5, 4: 1e-5, 6: 5e-6, 8: 1e-6, 
            10: 5e-7, 15: 1e-7, 20: 5e-8
        }
    if epoch in lr_adjust.keys():
        lr = lr_adjust[epoch]
        for param_group in optimizer.param_groups:
            param_group['lr'] = lr
        print('Updating learning rate to {}'.format(lr))

In [358]:
def get_data(args, flag):
        #rgs = self.args

        data_dict = {
            'train_norm':Dataset_Custom,
        }
        Data = data_dict[args.data]
        timeenc = 0 if args.embed!='timeF' else 1

        if flag == 'test':
            shuffle_flag = False; drop_last = True; batch_size = args.batch_size; freq=args.freq
        elif flag=='pred':
            shuffle_flag = False; drop_last = False; batch_size = 1; freq=args.detail_freq
            Data = Dataset_Pred
        else:
            shuffle_flag = True; drop_last = True; batch_size = args.batch_size; freq=args.freq
        data_set = Data(
            root_path=args.root_path,
            data_path=args.data_path,
            flag=flag,
            size=[args.seq_len, args.label_len, args.pred_len],
            features=args.features,
            target=args.target,
            inverse=args.inverse,
            timeenc=timeenc,
            freq=freq,
            cols=args.cols
        )
        data_loader = DataLoader(
            data_set,
            batch_size=batch_size,
            shuffle=shuffle_flag,
            num_workers=args.num_workers,
            drop_last=drop_last)

        return data_set, data_loader

In [ ]:
def select_criterion():
        criterion =  nn.MSELoss()
        return criterion

In [368]:
def select_optimizer():
        model_optim = optim.Adam(model.parameters(), lr=args.learning_rate)
        return model_optim

In [370]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch import optim

In [374]:
class Informer(nn.Module):
    def __init__(self, enc_in, dec_in, c_out, seq_len, label_len, out_len, 
                factor=5, d_model=512, n_heads=8, e_layers=3, d_layers=2, d_ff=512, 
                dropout=0.0, attn='prob', embed='fixed', freq='h', activation='gelu', 
                output_attention = False, distil=True, mix=True,
                device=torch.device('cuda:0')):
        super(Informer, self).__init__()
        self.pred_len = out_len
        self.attn = attn
        self.output_attention = output_attention

        # Encoding
        self.enc_embedding = DataEmbedding(enc_in, d_model, embed, freq, dropout)
        self.dec_embedding = DataEmbedding(dec_in, d_model, embed, freq, dropout)
        # Attention
        Attn = ProbAttention if attn=='prob' else FullAttention
        # Encoder
        self.encoder = Encoder(
            [
                EncoderLayer(
                    AttentionLayer(Attn(False, factor, attention_dropout=dropout, output_attention=output_attention), 
                                d_model, n_heads, mix=False),
                    d_model,
                    d_ff,
                    dropout=dropout,
                    activation=activation
                ) for l in range(e_layers)
            ],
            [
                ConvLayer(
                    d_model
                ) for l in range(e_layers-1)
            ] if distil else None,
            norm_layer=torch.nn.LayerNorm(d_model)
        )
        # Decoder
        self.decoder = Decoder(
            [
                DecoderLayer(
                    AttentionLayer(Attn(True, factor, attention_dropout=dropout, output_attention=False), 
                                d_model, n_heads, mix=mix),
                    AttentionLayer(FullAttention(False, factor, attention_dropout=dropout, output_attention=False), 
                                d_model, n_heads, mix=False),
                    d_model,
                    d_ff,
                    dropout=dropout,
                    activation=activation,
                )
                for l in range(d_layers)
            ],
            norm_layer=torch.nn.LayerNorm(d_model)
        )
        # self.end_conv1 = nn.Conv1d(in_channels=label_len+out_len, out_channels=out_len, kernel_size=1, bias=True)
        # self.end_conv2 = nn.Conv1d(in_channels=d_model, out_channels=c_out, kernel_size=1, bias=True)
        self.projection = nn.Linear(d_model, c_out, bias=True)
        
    def forward(self, x_enc, x_mark_enc, x_dec, x_mark_dec, 
                enc_self_mask=None, dec_self_mask=None, dec_enc_mask=None):
        enc_out = self.enc_embedding(x_enc, x_mark_enc)
        enc_out, attns = self.encoder(enc_out, attn_mask=enc_self_mask)

        dec_out = self.dec_embedding(x_dec, x_mark_dec)
        dec_out = self.decoder(dec_out, enc_out, x_mask=dec_self_mask, cross_mask=dec_enc_mask)
        dec_out = self.projection(dec_out)
        
        # dec_out = self.end_conv1(dec_out)
        # dec_out = self.end_conv2(dec_out.transpose(2,1)).transpose(1,2)
        if self.output_attention:
            return dec_out[:,-self.pred_len:,:], attns
        else:
            return dec_out[:,-self.pred_len:,:] # [B, L, D]

#### Code

In [391]:
Data=Dataset_Custom
flag

'val'

In [395]:
flag="train"
train_data = Data(
            root_path=args.root_path,
            data_path=args.data_path,
            flag=flag,
            size=[args.seq_len, args.label_len, args.pred_len],
            features=args.features,
            target=args.target,
            inverse=args.inverse,
            timeenc=timeenc,
            freq=args.freq,
            cols=args.cols
        )
train_loader = DataLoader(
            data_set,
            batch_size=batch_size,
            shuffle=shuffle_flag,
            num_workers=args.num_workers,
            drop_last=drop_last)

In [396]:
flag="val"
vali_data = Data(
            root_path=args.root_path,
            data_path=args.data_path,
            flag=flag,
            size=[args.seq_len, args.label_len, args.pred_len],
            features=args.features,
            target=args.target,
            inverse=args.inverse,
            timeenc=timeenc,
            freq=args.freq,
            cols=args.cols
        )
vali_loader = DataLoader(
            data_set,
            batch_size=batch_size,
            shuffle=shuffle_flag,
            num_workers=args.num_workers,
            drop_last=drop_last)

In [410]:
for i, (batch_x,batch_y,batch_x_mark,batch_y_mark) in enumerate(vali_loader):
    if i == 1:
        train_x_dataframe= pd.DataFrame(batch_x[0])

In [411]:
train_x_dataframe

,0,1,2,3,4,5,6,7,8,9,...,46,47,48,49,50,51,52,53,54,55
0,-0.274355,-2.107196,-0.684168,-0.654466,-1.638591,-0.573115,-1.269732,-0.664284,-1.800702,0.185467,...,-0.000052,-1.0,-0.000019,-0.000006,-0.000006,-0.00008,-0.000051,-0.001814,-0.00695,-0.637094
1,-0.274355,-2.107196,-0.684168,-0.522546,-1.638591,-0.526140,-0.600868,-0.664284,-1.917336,0.185467,...,-0.000052,-1.0,-0.000019,-0.000006,-0.000006,-0.00008,-0.000051,-0.001814,-0.00695,-0.478365
2,-0.274355,-1.059827,-0.677255,-0.522546,-0.635361,-0.526140,-0.600868,-0.468136,-1.917336,0.185467,...,-0.000052,-1.0,-0.000019,-0.000006,-0.000006,-0.00008,-0.000051,-0.001814,-0.00695,-0.478365
3,-0.274355,-1.059827,-0.677255,-0.605129,-0.635361,-0.552563,-1.269732,-0.468136,-1.323564,0.185467,...,-0.000052,-1.0,-0.000019,-0.000006,-0.000006,-0.00008,-0.000051,-0.001814,-0.00695,-0.617253
4,-0.274355,-0.270859,-0.497510,-0.605129,-0.133746,-0.552563,-1.269732,-0.666771,-1.323564,0.185467,...,-0.000052,-1.0,-0.000019,-0.000006,-0.000006,-0.00008,-0.000051,-0.001814,-0.00695,-0.617253
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91,-0.204419,-0.427187,-0.677255,-0.459520,-0.886169,-0.482618,-1.269732,-0.637948,0.419026,0.185467,...,-0.000052,-1.0,-0.000019,-0.000006,-0.000006,-0.00008,-0.000051,-0.001814,-0.00695,-0.147679
92,-0.194059,-0.735770,-0.497510,-0.459520,-0.635361,-0.482618,-1.269732,-0.653157,0.419026,0.185467,...,-0.000052,-1.0,-0.000019,-0.000006,-0.000006,-0.00008,-0.000051,-0.001814,-0.00695,-0.147679
93,-0.194059,-0.735770,-0.497510,-0.566110,-0.635361,-0.546272,-1.269732,-0.653157,-0.906069,0.185467,...,-0.000052,-1.0,-0.000019,-0.000006,-0.000006,-0.00008,-0.000051,-0.001814,-0.00695,-0.564343
94,-0.202088,-0.880360,-0.677255,-0.566110,-0.133746,-0.546272,-1.269732,-0.562680,-0.906069,0.185467,...,-0.000052,-1.0,-0.000019,-0.000006,-0.000006,-0.00008,-0.000051,-0.001814,-0.00695,-0.564343


In [413]:
# count the total number of NaN values in the dataframe
total_nan_count = train_x_dataframe.isna().sum().sum()
print("Nulos",total_nan_count)

0


In [414]:
# count the total number of zeros in the dataframe
total_zero_count = (train_x_dataframe == 0).sum().sum()
print(total_zero_count)

0


In [397]:
for i, (batch_x,batch_y,batch_x_mark,batch_y_mark) in enumerate(vali_loader):
    print(batch_x)

tensor([[[-2.1206e-01,  2.0439e+00,  1.5925e-01,  ..., -1.8138e-03,
          -6.9502e-03,  3.7291e-01],
         [-2.1206e-01,  2.0439e+00,  1.5925e-01,  ..., -1.8138e-03,
          -4.3811e+00,  1.1582e+00],
         [ 2.0611e-02,  3.0286e+00,  2.0604e+00,  ..., -1.8138e-03,
          -4.3811e+00,  1.1582e+00],
         ...,
         [-2.7435e-01, -5.1617e-01, -4.9751e-01,  ..., -1.8138e-03,
          -6.9502e-03, -4.6514e-01],
         [-1.8009e-01, -1.3986e+00, -1.3583e-02,  ..., -1.8138e-03,
          -6.9502e-03, -4.6514e-01],
         [-1.8009e-01, -1.3986e+00, -1.3583e-02,  ..., -1.8138e-03,
          -6.9502e-03, -5.7977e-01]],

        [[-2.7435e-01, -7.4947e-02, -6.7725e-01,  ..., -1.8138e-03,
          -6.9502e-03, -5.1805e-01],
         [-2.0660e-01, -1.1915e-02, -6.7725e-01,  ..., -1.8138e-03,
          -6.9502e-03, -5.1805e-01],
         [-2.0660e-01, -1.1915e-02, -6.7725e-01,  ..., -1.8138e-03,
          -6.9502e-03, -4.8718e-01],
         ...,
         [-1.9406e-01, -5

(277, 56)

In [331]:
data_set.data_x.shape

(277, 56)

In [184]:
true = batch_y[:,-args.pred_len:,f_dim:]
pred = batch_y[:,-args.pred_len:,:]
pred.shape

torch.Size([32, 24, 1])

In [315]:
border1 = len(df_raw)-args.seq_len
border2 = len(df_raw)
print(border1,border2)

1709 1805


In [219]:
d1=df_raw.values
d2=d1[border1:border2]
d2.shape

(96, 57)

In [162]:
# Es 35 por cada batch
train_steps = len(data_loader)
train_steps

35

In [163]:
data_loader

In [170]:
## ENTRADA DEL DECODER (Lo que va a predecir lo coloca en ceros)
dec_inp = torch.zeros([batch_y.shape[0],args.pred_len, batch_y.shape[-1]]).float()
dec_inp.shape

torch.Size([32, 24, 56])

In [173]:
dec_input = torch.cat([batch_y[:,:args.label_len,:], dec_inp], dim=1).float()
dec_input.shape

torch.Size([32, 72, 56])

In [380]:
data_loader.shape

AttributeError: 'DataLoader' object has no attribute 'shape'

In [378]:
for epoch in range(train_steps):
    for i, (batch_x,batch_y,batch_x_mark,batch_y_mark) in enumerate(data_loader):
        print(batch_y.shape)

torch.Size([32, 72, 56])
torch.Size([32, 72, 56])
torch.Size([32, 72, 56])
torch.Size([32, 72, 56])
torch.Size([32, 72, 56])
torch.Size([32, 72, 56])
torch.Size([32, 72, 56])
torch.Size([32, 72, 56])
torch.Size([32, 72, 56])
torch.Size([32, 72, 56])
torch.Size([32, 72, 56])
torch.Size([32, 72, 56])
torch.Size([32, 72, 56])
torch.Size([32, 72, 56])
torch.Size([32, 72, 56])
torch.Size([32, 72, 56])
torch.Size([32, 72, 56])
torch.Size([32, 72, 56])
torch.Size([32, 72, 56])
torch.Size([32, 72, 56])
torch.Size([32, 72, 56])
torch.Size([32, 72, 56])
torch.Size([32, 72, 56])
torch.Size([32, 72, 56])
torch.Size([32, 72, 56])
torch.Size([32, 72, 56])
torch.Size([32, 72, 56])
torch.Size([32, 72, 56])
torch.Size([32, 72, 56])
torch.Size([32, 72, 56])
torch.Size([32, 72, 56])
torch.Size([32, 72, 56])
torch.Size([32, 72, 56])
torch.Size([32, 72, 56])
torch.Size([32, 72, 56])
torch.Size([32, 72, 56])
torch.Size([32, 72, 56])
torch.Size([32, 72, 56])
torch.Size([32, 72, 56])
torch.Size([32, 72, 56])


In [326]:
# Filter the rows that have more than 0 zeros
filtered_rows = df.index[df.eq(0).sum(axis=1) > 0]

# Print the filtered rows
print(filtered_rows.shape)

(5,)


In [377]:
df_raw

,date,p50 deploy/adservice,p50 deploy/cartservice,p50 deploy/checkoutservice,p50 deploy/currencyservice,p50 deploy/emailservice,p50 deploy/frontend,p50 deploy/paymentservice,p50 deploy/productcatalogservice,p50 deploy/recommendationservice,...,deploy/cartservice,deploy/checkoutservice,deploy/currencyservice,deploy/emailservice,deploy/frontend,deploy/loadgenerator,deploy/productcatalogservice,deploy/recommendationservice,deploy/shippingservice,p95 deploy/frontend
0,2021-10-07 09:18:00,13.460422,-1.643790,-0.057770,-0.217764,-1.146121,-0.346990,-0.644470,-0.522305,0.705090,...,-0.000056,-0.000104,-0.000017,-0.000005,-0.000005,-0.000067,-0.000055,-0.001252,4.805699,-0.535875
1,2021-10-07 09:18:05,10.031784,0.229134,-0.434722,-0.217764,-0.898530,-0.346990,-0.644470,-0.429260,0.705090,...,-0.000056,-0.000104,-0.000017,-0.000005,-0.000005,-0.000067,-0.000055,-0.001252,4.805699,-0.535875
2,2021-10-07 09:18:10,10.031784,0.229134,-0.434722,-0.228521,-0.898530,-0.393260,0.922269,-0.429260,-0.498528,...,-0.000056,-0.000104,-0.000017,-0.000005,-0.000005,-0.000067,-0.000055,-0.001252,-0.005351,-0.623315
3,2021-10-07 09:18:15,8.460325,-0.890356,-0.010651,-0.228521,-0.452866,-0.393260,0.922269,-0.592773,-0.498528,...,-0.000056,-0.000104,-0.000017,-0.000005,-0.000005,-0.000067,-0.000055,-0.001252,-0.005351,-0.623315
4,2021-10-07 09:18:20,8.460325,-0.890356,-0.010651,-0.325898,-0.452866,-0.392411,0.400022,-0.592773,-0.798506,...,-0.000056,-0.000104,-0.000017,-0.000005,-0.000005,-0.000067,-0.000055,-0.001252,-0.005351,-0.237772
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1800,2021-10-07 11:48:00,-0.254131,0.002309,-0.010651,-0.722205,0.587018,-0.532381,0.086675,-0.670068,-0.714562,...,-0.000056,-0.000104,-0.000017,-0.000005,-0.000005,-0.000067,-0.000055,-2.177623,-0.005351,-0.463400
1801,2021-10-07 11:48:05,-0.254131,-0.166355,-0.434722,-0.722205,0.215631,-0.532381,0.086675,-0.586403,-0.714562,...,-0.000056,-0.000104,-0.000017,-0.000005,-0.000005,-0.000067,-0.000055,-2.177623,-0.005351,-0.463400
1802,2021-10-07 11:48:10,-0.254131,-0.166355,-0.434722,0.248105,0.215631,-0.503038,-0.748919,-0.586403,-0.665183,...,-0.000056,-0.000104,-0.000017,-0.000005,-0.000005,-0.000067,-0.000055,-0.001252,-0.005351,0.627135
1803,2021-10-07 11:48:15,0.031589,0.688184,0.554777,0.248105,-0.650939,-0.503038,-0.748919,-0.471991,-0.665183,...,-0.000056,-0.000104,-0.000017,-0.000005,-0.000005,-0.000067,-0.000055,-0.001252,-0.005351,0.627135


## Pred

In [288]:
def train(self, setting):
        train_data, train_loader = self._get_data(flag = 'train')
        vali_data, vali_loader = self._get_data(flag = 'val')
        test_data, test_loader = self._get_data(flag = 'test')

        path = os.path.join(self.args.checkpoints, setting)
        if not os.path.exists(path):
            os.makedirs(path)

        time_now = time.time()
        
        train_steps = len(train_loader)
        early_stopping = EarlyStopping(patience=self.args.patience, verbose=True)
        
        model_optim = self._select_optimizer()
        criterion =  self._select_criterion()

        if self.args.use_amp:
            scaler = torch.cuda.amp.GradScaler()

        for epoch in range(self.args.train_epochs):
            iter_count = 0
            train_loss = []
            
            self.model.train()
            epoch_time = time.time()
            for i, (batch_x,batch_y,batch_x_mark,batch_y_mark) in enumerate(train_loader):
                iter_count += 1
                
                model_optim.zero_grad()
                pred, true = self._process_one_batch(
                    train_data, batch_x, batch_y, batch_x_mark, batch_y_mark)
                print(pred)
                loss = criterion(pred, true)
                train_loss.append(loss.item())
                
                if (i+1) % 100==0:
                    print("\titers: {0}, epoch: {1} | loss: {2:.7f}".format(i + 1, epoch + 1, loss.item()))
                    speed = (time.time()-time_now)/iter_count
                    left_time = speed*((self.args.train_epochs - epoch)*train_steps - i)
                    print('\tspeed: {:.4f}s/iter; left time: {:.4f}s'.format(speed, left_time))
                    iter_count = 0
                    time_now = time.time()
                
                if self.args.use_amp:
                    scaler.scale(loss).backward()
                    scaler.step(model_optim)
                    scaler.update()
                else:
                    loss.backward()
                    model_optim.step()

            print("Epoch: {} cost time: {}".format(epoch+1, time.time()-epoch_time))
            train_loss = np.average(train_loss)
            vali_loss = self.vali(vali_data, vali_loader, criterion)
            test_loss = self.vali(test_data, test_loader, criterion)

            print("Epoch: {0}, Steps: {1} | Train Loss: {2:.7f} Vali Loss: {3:.7f} Test Loss: {4:.7f}".format(
                epoch + 1, train_steps, train_loss, vali_loss, test_loss))
            early_stopping(vali_loss, self.model, path)
            if early_stopping.early_stop:
                print("Early stopping")
                break

            adjust_learning_rate(model_optim, epoch+1, self.args)
            
        best_model_path = path+'/'+'checkpoint.pth'
        self.model.load_state_dict(torch.load(best_model_path))
        
        return self.model

In [290]:
vali_data, vali_loader = get_data(flag = 'val')

TypeError: get_data() missing 1 required positional argument: 'args'